In [10]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
# Database connection and query
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

df_clients = pd.read_sql_query("SELECT * FROM core_client", conn)
df_sales = pd.read_sql("SELECT * FROM core_sale;", conn)
df_salesmans = pd.read_sql("SELECT * FROM core_salesman;", conn)
df_categories = pd.read_sql("SELECT * FROM core_category;", conn)
df_products = pd.read_sql("SELECT * FROM core_product;", conn)
df_productsales = pd.read_sql("SELECT * FROM core_productsale;", conn)
df_productsales

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn

,id,quantity,income,product_id,sale_id,income_currency
0,16921,4.10,14.35,17,00034279,USD
1,16953,4.00,14.00,17,00034290,USD
2,16930,8.10,43.52,22,00034282,USD
3,17005,6.80,21.87,23,00034311,USD
4,16906,24.00,46.72,33,00034269,USD
...,...,...,...,...,...,...
71828,18772,25.30,0.00,489,00034825,USD
71829,19432,4.30,0.00,489,00035023,USD
71830,19630,8.70,0.00,489,00035079,USD
71831,59878,4.40,0.00,489,00001021,USD


In [12]:
# Quantity of sales a salesman made with every client

df_merged = pd.merge(df_sales, df_salesmans, left_on='salesman_id', right_on='id', suffixes=('_sale', '_salesman'))
df_merged_clients = pd.merge(df_merged, df_clients, left_on='client_id', right_on='id', suffixes=('_sale', '_client'))
df_merged_clients.drop(columns=['id_sale','id_salesman','date', 'status', 'income_currency', 'description', 'identity_card_sale', 'image_sale', 'phone_1', 'phone_2', 'address_sale', 'identity_card_client', 'phone', 'address_client', 'image_client'], inplace=True)
df_merged_clients.rename(columns={'name_sale': 'name_salesman'}, inplace=True)
df_merged_clients.set_index(['salesman_id', 'client_id', 'name_client'], inplace=True)
df_merged_clients["count"] = df_merged_clients.groupby(['salesman_id' ,'client_id'])['id'].count()
df_merged_clients["total_income"] = df_merged_clients.groupby(['salesman_id' ,'client_id'])['income'].sum()
df_merged_clients.reset_index(inplace=True)
data = {
    "name_salesman": df_merged_clients.groupby(['salesman_id' ,'client_id'])['name_salesman'].first(),
    "name_client": df_merged_clients.groupby(['salesman_id' ,'client_id'])['name_client'].first(),
    "count": df_merged_clients.groupby(['salesman_id' ,'client_id'])['count'].first(),
    'total_income': df_merged_clients.groupby(['salesman_id' ,'client_id'])['total_income'].first()
}
df_aux = pd.DataFrame(data)
# df_aux.reset_index(inplace=True)
df_aux

name_salesman  \
salesman_id client_id                   
1           2          VENTAS DIRECTA   
            3          VENTAS DIRECTA   
            5          VENTAS DIRECTA   
            8          VENTAS DIRECTA   
            9          VENTAS DIRECTA   
...                               ...   
14          1034         Gunner Oviol   
            1061         Gunner Oviol   
15          32               inactivo   
            320              inactivo   
            961              inactivo   

                                                             name_client  \
salesman_id client_id                                                      
1           2                                           VECINO DEFREITAS   
            3                                                     KALIFA   
            5                                            VENTAS DIRECTAS   
            8                                VICTOR JOSE NAVEDA PARTIDAS   
            9                              RAMIRO JOSE VILLALOBOS BLANCO   
...                                                                  ...   
14          1034       CELIA VICTORIA COLINA BRAVO (DANNAÑS MARKET DE...   
            1061                     MARIA AUXILIADORA JELABI DE BEUVIRN   
15          32                     DAVIDE DA CONCEICAO NUNES (AVEIRENSE)   
            320        PANADERIA Y PASTELERIA EL RINCON DEL TACHIRA J...   
            961        FRANK ROBERT ROMERO SECO (ABASTO Y LICORERIA M...   

                       count  total_income  
salesman_id client_id                       
1           2             16       6900.48  
            3              1         10.90  
            5             36     136310.72  
            8              1         11.44  
            9              8         95.01  
...                      ...           ...  
14          1034           5        169.54  
            1061           2        148.34  
15          32             1         40.51  
            320            3         20.76  
            961            1         42.19  

[1499 rows x 4 columns]

In [21]:
# Quantity and income for categories for a salesman

df_merged_sales = pd.merge(df_sales, df_productsales, left_on='id', right_on='sale_id', suffixes=('_sale', '_productsales'))
df_merged_with_products = pd.merge(df_merged_sales, df_products, left_on='product_id', right_on='id', suffixes=('_sales', '_products'))
df_merged_with_categories = pd.merge(df_merged_with_products, df_categories, left_on='category_id', right_on='id', suffixes=('_products', '_categories'))
df_final_merged = pd.merge(df_merged_with_categories, df_salesmans, left_on='salesman_id', right_on='id', suffixes=('_categories', '_client'))
df_final_merged.drop(columns=['date', 'description_sales', 'income_currency_sale', 'status', 
                    'price_1_currency', 'price_2_currency','price_3_currency',
                    'price_1', 'price_2', 'price_3',
                    'code', 'identity_card', 'image_client',
                    'address', 'presentation',
                    'income_currency_productsales', 'cost',
                    'brand', 'cost_currency', 'description_products',
                    'image_categories', 'id', 'id_categories',
                    'income_sale', 'id_sale', 'id_products',
                    'client_id'
                    ], inplace=True)
df_final_merged.set_index(['salesman_id', 'category_id'], inplace=True)
df_final_merged['count'] = df_final_merged.groupby(['salesman_id', 'category_id'])['quantity'].sum()
df_final_merged['total_income'] = df_final_merged.groupby(['salesman_id', 'category_id'])['income_productsales'].sum()
df_final_merged.reset_index(inplace=True)
data = {
    "name_salesman": df_final_merged.groupby(['salesman_id', 'category_id'])['name'].first(),
    "name_category": df_final_merged.groupby(['salesman_id', 'category_id'])['name_categories'].first(),
    "salesman_id": df_final_merged.groupby(['salesman_id', 'category_id'])['salesman_id'].first(),
    "category_id": df_final_merged.groupby(['salesman_id', 'category_id'])['category_id'].first(),
    "count": df_final_merged.groupby(['salesman_id', 'category_id'])['count'].first(),
    "total_income": df_final_merged.groupby(['salesman_id', 'category_id'])['total_income'].first()
}

df_aux_2 = pd.DataFrame(data)
df_aux_2.drop(columns=['category_id', 'salesman_id'], inplace=True)
df_aux_2.reset_index(inplace=True)
df_aux_2

,salesman_id,category_id,name_salesman,name_category,count,total_income
0,1,8,VENTAS DIRECTA,CREMAS Y NATILLAS,7313.00,15264.29
1,1,16,VENTAS DIRECTA,CHORIZOS,130.25,858.13
2,1,17,VENTAS DIRECTA,JAMONERIA,3434.17,20729.16
3,1,19,VENTAS DIRECTA,BOLOGNAS Y FIAMBRES,12650.74,52818.89
4,1,20,VENTAS DIRECTA,PAVOS Y POLLOS,235.81,1772.89
...,...,...,...,...,...,...
451,14,1064,Gunner Oviol,CONFITERIA,44.00,222.68
452,15,19,inactivo,BOLOGNAS Y FIAMBRES,3.67,15.39
453,15,23,inactivo,QUESOS FRESCOS,6.42,40.51
454,15,45,inactivo,CHOCOLATES Y GOLOSINAS,9.00,42.19


In [23]:
# Quantity and income for products for a salesman

df_merged_sales = pd.merge(df_sales, df_productsales, left_on='id', right_on='sale_id', suffixes=('_sale', '_productsales'))
df_merged_with_products = pd.merge(df_merged_sales, df_products, left_on='product_id', right_on='id', suffixes=('_sales', '_products'))
df_final_merged = pd.merge(df_merged_with_products, df_salesmans, left_on='salesman_id', right_on='id', suffixes=('_sales_products', '_salesmans'))
df_final_merged.drop(columns=['date', 'description_sales', 'income_currency_sale', 'status', 
                    'price_1_currency', 'price_2_currency','price_3_currency',
                    'price_1', 'price_2', 'price_3',
                    'code', 'identity_card',
                    'address', 'presentation',
                    'income_currency_productsales', 'cost',
                    'brand', 'cost_currency', 'description_products',
                    'income_sale', 'id_sale', 'image_sales_products',
                    'category_id', 'image_salesmans', 'phone_1', 'phone_2', 'client_id'
                    ], inplace=True)
df_final_merged.set_index(['salesman_id', 'product_id'], inplace=True)
df_final_merged['count'] = df_final_merged.groupby(['salesman_id', 'product_id'])['quantity'].sum()
df_final_merged['total_income'] = df_final_merged.groupby(['salesman_id', 'product_id'])['income_productsales'].sum()
df_final_merged.rename(columns={'name_sales_products':'name_products'}, inplace=True)

df_final_merged.reset_index(inplace=True)
df_final_merged[df_final_merged['salesman_id'] == 1]


,salesman_id,product_id,id_productsales,quantity,income_productsales,sale_id,id_sales_products,name_products,id_salesmans,name_salesmans,count,total_income
21796,1,489,17097,17.20,0.00,00034347,489,NONE,1,VENTAS DIRECTA,1331.93,0.00
21797,1,489,17002,25.72,0.00,00034311,489,NONE,1,VENTAS DIRECTA,1331.93,0.00
21798,1,489,17520,72.59,0.00,00034492,489,NONE,1,VENTAS DIRECTA,1331.93,0.00
21799,1,489,17589,120.00,0.00,00034512,489,NONE,1,VENTAS DIRECTA,1331.93,0.00
21800,1,489,17590,150.00,0.00,00034512,489,NONE,1,VENTAS DIRECTA,1331.93,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
26580,1,6,81979,11.00,30.25,NE010339,6,ACEITUNAS RE/PIM ENV. 450GR ESCURRIDO,1,VENTAS DIRECTA,11.00,30.25
26581,1,7,81980,5.00,0.00,NE010339,7,ALCAPARRAS ENVASADAS,1,VENTAS DIRECTA,5.00,0.00
26582,1,247,61024,156.60,481.28,00001746,247,POLLO ESPECIAL DE PRIMERA DEL CORRAL,1,VENTAS DIRECTA,156.60,481.28
26583,1,248,61026,55.80,197.35,00001746,248,CUARTO TRASERO (GRANEL ),1,VENTAS DIRECTA,55.80,197.35
